In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
df = pd.read_csv(r"../input/up-crime-data-year-2019/MR data - Compiled Data Set.csv")
df.head()

In [ ]:
import string
def clean(s):
    s = s.lower()
    s = s.replace('ghaziabad:', '')
    s = s.replace('lucknow:', '')
    s = s.replace('uttar pradesh:', '')
    s = s.replace('up:', '')
    s = s.replace('kanpur:', '')
    s = s.replace("-"," ")
    s = re.sub('[^\W*\w+\W*][^a-zA-Z0-9\s]', '', s)
    s = s.strip()
    p = set(string.punctuation)
    words = s.split()
    c_text=[]
    for i in range(10): #adding numbers from 0 - 9 to p 
        p.add(str(i))
    for i in words:
        t=''.join([x for x in i.encode("ascii","ignore").decode("ascii") if x not in p]) # ignoring non ascii charecters and numbers
        c_text.append(t)
    return " ".join([i for i in c_text])

In [ ]:
title = df.Title.values

In [ ]:
title_cleaned = []
for i in title:
    title_cleaned.append(clean(i))

In [ ]:
title_cleaned[0:10]

In [ ]:
df['title_cleaned'] = title_cleaned
text = df['Text'].values

In [ ]:
text_cleaned = []
for i in text:
    text_cleaned.append(clean(i))

In [ ]:
print(text_cleaned[:1])

In [ ]:
df['text_cleaned'] = text_cleaned

In [ ]:
df.head()

In [ ]:
df = df.rename(columns = {'Murder Happened with reason:\n1:Property/Land Disputes,\n2:Family Dispute\n3:Petty Quarrels,\n4:Money Disputes, \n5:Personal Vendetta, \n6:Love Affairs, \n7:Casteism\n8: Unknown/other':'Murder Reason','Crime Against Women ( Combined ):\n1. Murder with Rape \n2. Dowry Deaths(Sec. 3048)\n3. Suicide(sec 305/306)\n4. Kidnapping(All)\n5. Acid Attack(Sec. 326A IPC)\n6. Cruelty by Husband/in-laws((Sec.498 A IPC)\n7. Rape only(Sec. 376 or 511 IPC)\n8. Assault on Women with Intent to Outrage her Modesty (Sec. 354 IPC)\n9. Cyber Crimes against Women\n10. Protection of Children from Sexual Offences Act':'Crime Against Women'}, inplace=False)
df.head()

In [ ]:
df.columns

In [ ]:
clean_df = pd.read_csv(r"../input/cleaned-data-prefinal/MR data1.csv",encoding='utf-8-sig', engine='python')

In [ ]:
clean_df.columns.str.strip()

In [ ]:
clean_df.head()

In [ ]:
clean_df.groupby(["City"]).sum()

In [ ]:
clean_df['Total Victims'] = clean_df['Child Victims'] + clean_df['Male Victim'] + clean_df['Female Victim']

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
columns = ['Child Victims', 'Male Victim', 'Female Victim']
for i in columns:
    fig = px.treemap(clean_df,values=i,path=['City'],template="plotly_dark",title="<b>TreeMap representation of different city w.r.t. their {}</b>".format(i))
    fig.show()

In [ ]:
from textblob import TextBlob

In [ ]:
polarity = []
for i in clean_df['Title']:
    polarity.append(TextBlob(i).sentiment.polarity)

In [ ]:
clean_df['polarity'] = polarity

In [ ]:
clean_df.head()

In [ ]:
from wordcloud import WordCloud,STOPWORDS
stopwords = set(STOPWORDS)
total_title = ' '.join(clean_df['Title'])

In [ ]:
wordcloud = WordCloud(width=2000,height=1000,stopwords=stopwords).generate(total_title)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud)
plt.axis('off')

In [ ]:
clean_df.describe()

In [ ]:
data = pd.read_csv(r"../input/cleaned-data-up-crime/MR data 2.csv",encoding='utf-8-sig', engine='python')
data.head()

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
nltk.download('punkt')

In [ ]:
text = data.Text.values
#stemming
from nltk.stem.snowball import SnowballStemmer
s = SnowballStemmer("english")
text_stem = []
for i in text:
    words = i.split()
    stem = [s.stem(j) for j in words]
    stem_final = ' '.join(stem)
    text_stem.append(stem_final.split())

In [ ]:
from nltk.corpus import stopwords

In [ ]:
# Tokenizing the text
stop = set(stopwords.words("english"))
text_stop = []
for text in text_stem:
    remove = ' '.join(word for word in text if word not in stop)
    text_stop.append(remove.strip())

In [ ]:
data['Text_cleaned'] = text_stop

In [ ]:
data.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(data.Text_cleaned.values)
X = x.toarray()

In [ ]:
from sklearn.cluster import KMeans

NUMBER_OF_CLUSTERS = 3
km = KMeans(
    n_clusters=NUMBER_OF_CLUSTERS, 
    init='k-means++', 
    max_iter=500)
km.fit(X)

In [ ]:
from sklearn.decomposition import PCA

# First: for every document we get its corresponding cluster
clusters = km.predict(X)

# We train the PCA on the dense version of the tf-idf. 
pca = PCA(n_components=2)
two_dim = pca.fit_transform(X)

scatter_x = two_dim[:, 0] # first principle component
scatter_y = two_dim[:, 1] # second principle component

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')

fig, ax = plt.subplots()
fig.set_size_inches(20,10)

# color map for NUMBER_OF_CLUSTERS we have
cmap = {0: 'green', 1: 'blue', 2: 'red'}

# group by clusters and scatter plot every cluster
# with a colour and a label
for group in np.unique(clusters):
    ix = np.where(clusters == group)
    ax.scatter(scatter_x[ix], scatter_y[ix], c=cmap[group], label=group)

ax.legend()
plt.xlabel("PCA 0")
plt.ylabel("PCA 1")
plt.show()

In [ ]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

terms = v.get_feature_names()
for i in range(3):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :50]:
        print(' %s' % terms[ind], end='')
    print()